In [3]:
import pandas as pd
import keras
import tensorflow as tf
import random
import decimal
import json
from random import randrange, uniform
from sklearn import preprocessing
import keras.backend as K
from keras.utils.vis_utils import plot_model




MAX_TIME =  30000 * 3008 

TASK_NAME_LIST = ['mProject', 'mDiffFit', 'mConcatFit', 'mBgModel', 'mBackground', 'mImgtbl', 'mAdd', 'mViewer']

RAM_MAP = {128:30.0,
           192:39.0,
           256:6.0,
           320:28.0,
           384:36.0,
           448:11.0,
           512:18.0,
           576:9.0,
           640:32.0,
           704:17.0,
           768:3.0,
           832:8.0,
           896:24.0,
           960:34.0,
           1024:1.0,
           1088:44.0,
           1152:16.0,
           1216:13.0,
           1280:12.0,
           1344:27.0,
           1408:33.0,
           1472:14.0,
           1536:22.0,
           1600:7.0,
           1664:20.0,
           1728:37.0,
           1792:40.0,
           1856:35.0,
           1920:41.0,
           1984:46.0,
           2048:29.0,
           2112:21.0,
           2176:23.0,
           2240:26.0,
           2304:25.0,
           2368:2.0,
           2432:45.0,
           2496:43.0,
           2560:10.0,
           2624:42.0,
           2688:31.0,
           2752:4.0,
           2816:38.0,
           2880:5.0,
           2944:15.0,
           3008:19.0}

TASK_MIN_RAM = {
    'mProject': 896,
    'mDiffFit': 128, 
    'mConcatFit': 192,
    'mBgModel': 128,
    'mBackground': 128,
    'mImgtbl':320,
    'mAdd': 384,
    'mViewer':128}

H_TASK_MIN_RAM = {
    'mProject': 1024,
    'mDiffFit': 128, 
    'mConcatFit': 192,
    'mBgModel': 1280,
    'mBackground': 192,
    'mImgtbl':512,
    'mAdd': 512,
    'mViewer':320}

In [2]:
def map_ram(ram):
    return RAM_MAP[ram]

In [3]:
def map_task_type(task_type_name):
    return TASK_NAME_LIST.index(task_type_name)

In [4]:
def get_previous_task_type(task_name):
    return TASK_NAME_LIST[TASK_NAME_LIST.index(task_name) -1]

In [5]:
def next_ram(ram):
    ram_list = list(range(128, 3072 ,64))
    cur_index = ram_list.index(ram)
    if cur_index == 45:
        return 3008
    else:
        return ram_list[cur_index + 1]

In [6]:
def previous_ram(ram):
    ram_list = list(range(128, 3072 ,64))
    cur_index = ram_list.index(ram)
    if cur_index == 0:
        return 128
    else:
        return ram_list[cur_index - 1]

In [7]:
normalize = (28637.0 - 169.0) + 169.0
0.010518 * normalize

301.203966

In [8]:
def getPredictedExecutionTime(ram, task_type_name, workflow_size, df_x, df_x_failed, model, model_failed, iteration, mview, hyperflow_model):
        
    if hyperflow_model:
        normalize = (70063.0 - 355.0)
        normalize_add = 355
    else:
        normalize = (28637.0 - 169.0) 
        normalize_add = 169.0

    in_from = -1
    in_to = 1
    
    if task_type_name == 'mViewer':
        if mview:
            in_from = 0.00001
            in_to = 1
        else:
            in_from = 0
            in_to = 0
            
    mapped_ram_normalized = (ram - 128) / ( 3008 - 128)
    #print(mapped_ram_normalized)
    mapped_ram = map_ram(ram)
    mapped_task_type_normalized = round(map_task_type(task_type_name) / 7, 8)
    #print(mapped_task_type_normalized)
    mapped_task_type = map_task_type(task_type_name)
    #print(df_x_failed[(df_x_failed['ram_mapped'] == mapped_ram) & (df_x_failed['task_type'] == mapped_task_type) & (df_x_failed['workflow_size_mapped'] == workflow_size)].index.values)
    ids_failed = list(df_x_failed[(abs(df_x_failed['ram'] - mapped_ram_normalized) < 0.0001)  & 
                                  (abs(df_x_failed['task_type'] - mapped_task_type_normalized) < 0.0001) & 
                                  (abs(df_x_failed['workflow_size_mapped'] - workflow_size) < 0.0001)].index.values)
    #print(df_x[(df_x['ram_mapped'] == mapped_ram) & (df_x['task_type'] == mapped_task_type) & (df_x['workflow_size_mapped'] == workflow_size)])
    #print(df_x[abs(df_x['task_type'] - mapped_task_type_normalized) < 0.0001])
    
    ids = list(df_x[(abs(df_x['ram'] - mapped_ram_normalized) < 0.0001 ) & 
                    (abs(df_x['task_type'] - mapped_task_type_normalized) < 0.0001) & 
                    (df_x['workflow_size_mapped'] == workflow_size) &
                    (df_x['count_in_ln'] >= in_from) & 
                    (df_x['count_in_ln'] <= in_to)].index.values) 
    
    #print(ids)
    failed_chance = model_failed.predict(df_x_failed.loc[[random.choice(ids_failed)]].values)[0][0]
    if task_type_name == 'mBgModel':
        if ram < 1153:
            failed_chance = failed_chance + 0.7
        else:
            failed_chance = failed_chance - 0.3
    if (failed_chance > 0.7) or (task_type_name == 'mConcatFit' and ram == 128):
        return MAX_TIME / ram
    if (len(ids) == 0):
        return MAX_TIME / ram  
    if H_TASK_MIN_RAM[task_type_name] > ram:
        return MAX_TIME / ram
    else:
        value = df_x.loc[[random.choice(ids)]]
        
        predicted_value = model.predict(value.values)[0][0]
#         if task_type_name == 'mAdd' or task_type_name == 'mBackground' or task_type_name == 'mBgModel':
#             perc = random.randint(5, 10)
#             predicted_value = predicted_value * ( 100 + perc) / 100
        return predicted_value * normalize + normalize_add
    

In [15]:
model = tf.keras.models.load_model('./best-model-hyperflow-final-new', compile = False)
model.layers[0].input_shape

AttributeError: The layer has never been called and thus has no defined input shape.

In [9]:
failed_model = tf.keras.models.load_model('./best-failed-model-normalized-true')
model = tf.keras.models.load_model('./best-model-hyperflow-final-new-128')
hyperflow_model = tf.keras.models.load_model('./best-model-hyperflow-final-new-hyperflow-128')
for ram in list(range(128, 3072 ,64)):
    print("RAM " + str(ram))
    print(getPredictedExecutionTime(ram, 'mProject', 0, get_x(), get_x_failed(), model, failed_model, 0, True, False))
    print(getPredictedExecutionTime(ram, 'mProject', 0, get_x(), get_x_failed(), hyperflow_model, failed_model, 0, True, True))
    print("==========")

KeyboardInterrupt: 

In [ ]:
2439.8923255205154
2679.3280113339424

In [22]:
def decorate_workflow(workflow):
    df_x = get_x()
    df_x_failed = get_x_failed()
    model = tf.keras.models.load_model('./best-model-hyperflow-final-new-512')
    hyperflow_model = tf.keras.models.load_model('./best-model-hyperflow-final-new-hyperflow-512')

    failed_model = tf.keras.models.load_model('./best-failed-model-normalized')
    tasks = workflow['tasks']
    max_tasks_start_times = {
        'mProject': {}, 
        'mDiffFit': {}, 
        'mConcatFit': {}, 
        'mBgModel': {}, 
        'mBackground': {}, 
        'mImgtbl': {}, 
        'mAdd': {}, 
        'mViewer': {}    }
    min_tasks_finish_times = {
        'mProject': {}, 
        'mDiffFit': {}, 
        'mConcatFit': {}, 
        'mBgModel': {}, 
        'mBackground': {}, 
        'mImgtbl': {}, 
        'mAdd': {}, 
        'mViewer': {}    }
    for task_type in TASK_NAME_LIST:
        for ram in list(range(128, 3072 ,64)):
            max_tasks_start_times[task_type][str(ram)] = 0
            min_tasks_finish_times[task_type][str(ram)] = 99999999
    mdiff_times = {}
    mdiff_hyperflow_times = {}

    whole_calculation = len(workflow['tasks']) * len(list(range(128, 3072 ,64)))
    current_calculation = 0
    for task_type in TASK_NAME_LIST:
        print(task_type)
        inc_time = 0
        for task in workflow['tasks']:
            if task['name'] == task_type:
                inc_time += 6
                for ram in list(range(128, 3072 ,64)):
                            
                    current_calculation += 1
                    current_task_predecessors = find_predecessor(workflow, task)
                    mview = False
                    if len(task['ins']) > 1:
                        mview = True

                    time = getPredictedExecutionTime(ram, task['name'], 0.0, df_x, df_x_failed, model, failed_model, 0, mview, False)
                    hyperflow_time = getPredictedExecutionTime(ram, task['name'], 0.0, df_x, df_x_failed, hyperflow_model, failed_model, 0, mview, True)
                    delay = max(10, hyperflow_time - time)
                    if 'hyperflowTime' not in task:
                        task['hyperflowTime'] = {}
                    task['hyperflowTime'][str(ram)] = hyperflow_time
                    finish_times = []
                    if 'startTime' not in task:
                        task['startTime'] = {}
                    if 'finishTime' not in task:
                        task['finishTime'] = {}
                    for predecessor in current_task_predecessors:
                        finish_times.append(predecessor['finishTime'][str(ram)])
                    if len(current_task_predecessors) == 0:
                        task['startTime'][str(ram)] = delay + inc_time
                        task['finishTime'][str(ram)] = delay + inc_time + int(time)
                    else:   
                        max_finish_time = max(finish_times)
                        start_time = delay + max(inc_time + max_finish_time, max_tasks_start_times[get_previous_task_type(task_type)][str(ram)] + 20)
                        task['startTime'][str(ram)] =  start_time
                        task['finishTime'][str(ram)] = start_time + int(time)
                    min_tasks_finish_times[task_type][str(ram)] = min(task['finishTime'][str(ram)], min_tasks_finish_times[task_type][str(ram)])
                    max_tasks_start_times[task_type][str(ram)] = max(task['startTime'][str(ram)], max_tasks_start_times[task_type][str(ram)])
                    
    with open('./generated-workflow-512.json', 'w+') as result_json:
        json.dump(workflow, result_json, indent=4)
                        

In [10]:
def get_x():
    df_results = pd.read_csv('./montage-final.csv')
    df_results = df_results[df_results['successed'] == 0]
    del df_results['task_id']
    del df_results['workflow']

    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(df_results.values)
    df_results = pd.DataFrame(x_scaled, columns=df_results.columns)
    df_y = df_results['lambda_time']
    del df_results['lambda_time_100ms_rounded']
    del df_results['download_time_100ms_rounded_normalized']
    del df_results['ram_mapped']
    del df_results['lambda_time']
    del df_results['execution_time_100ms_rounded']
    del df_results['execution_time_100ms_rounded_normalized']
    del df_results['upload_time_100ms_rounded']
    del df_results['upload_time_100ms_rounded_normalized']
    del df_results['download_time_100ms_rounded']
    del df_results['successed']
    del df_results['lambda_time_100ms_rounded_normalized']
    del df_results['workflow_size']
    del df_results['iteration']
    del df_results['Unnamed: 0']
    df_results = df_results.reset_index(drop=True)
    return df_results

In [11]:
sum([1,2,3]) 

6

In [12]:
def get_x_with_time():
    df_core = pd.read_csv('./results-montage.csv')
    df_more_data = pd.read_csv('./montage-more-data.csv')
    df_results = pd.concat([df_core, df_more_data])
    df_results = df_results[df_results['successed'] == 0]
    del df_results['download_time_100ms_rounded_normalized']
    del df_results['ram']
    del df_results['execution_time_100ms_rounded']
    del df_results['execution_time_100ms_rounded_normalized']
    del df_results['upload_time_100ms_rounded']
    del df_results['upload_time_100ms_rounded_normalized']
    del df_results['download_time_100ms_rounded']
    del df_results['task_id']
    del df_results['successed']
    del df_results['lambda_time_100ms_rounded_normalized']
    del df_results['workflow']
    del df_results['workflow_size']
    return df_results

In [13]:
def get_x_failed():
# DO ZMIANY
    df_results = pd.read_csv('./x_failed_normalized.csv')
    return df_results

In [14]:
def find_predecessor(workflow, the_task):
    predecessors = []
    for task in workflow['tasks']:
        if any(x in task['outs'] for x in the_task['ins']):
            predecessors.append(task)
    return predecessors

In [15]:
def decorate_workflow_all_ram(data):
    for ram in list(range(128, 3072 ,64)):
        print(ram)
        decorate_workflow(data, ram)
    with open('./generated-workflow.json', 'w+') as result_json:
        json.dump(data, result_json, indent=4)

In [16]:
def fix_generated_times(task):
    for ram in list(range(128, 3072 ,64)):
        start_time = task['startTime'][str(ram)]
        end_time = task['finishTime'][str(ram)]
        total_time = end_time - start_time
        previous_ram_total_time = get_total_time(task, previous_ram(ram))
        next_ram_total_time = get_total_time(task, next_ram(ram))

        if (total_time < (0.9 * min(previous_ram_total_time, next_ram_total_time))) or (total_time > (1.1 * max(previous_ram_total_time, next_ram_total_time))):
            task['finishTime'][str(ram)] = int(start_time + (previous_ram_total_time + next_ram_total_time)/2)

In [17]:
def get_total_time(task, ram):
    start_time = task['startTime'][str(ram)]
    end_time = task['finishTime'][str(ram)]
    return end_time - start_time

In [18]:
def find_predecessor(workflow, the_task):
    predecessors = []
    for task in workflow['tasks']:
        if any(x in task['outs'] for x in the_task['ins']):
            predecessors.append(task)
    return predecessors

In [58]:
df = get_x()
df[df['task_type'] == 3][df['ram_mapped'] == 30]

KeyError: 'ram_mapped'

In [59]:
df_failed = get_x_failed()
df_x_failed[(df_x['ram_mapped'] == mapped_ram) & (df_x['task_type'] == mapped_task_type) & (df_x['workflow_size_mapped'] == workflow_size)] 


NameError: name 'df_x_failed' is not defined

In [23]:
with open('./workflow.json') as result_json:
    data = json.load(result_json)
decorate_workflow(data)

mProject
mDiffFit
mConcatFit
mBgModel
mBackground
mImgtbl
mAdd
mViewer


In [19]:
values = [(0.56, 0.76), (0.62, 1.25), (1.61, 1.91)]

In [ ]:
final = []
for i in range (0, len(values)):
    for j in range (0, len(values)):
        

In [88]:
with open('./generated-workflow.json') as result_json:
    data = json.load(result_json)
for task in data['tasks']:
    for ram in list(range(128, 3072 ,64)):
        if ram % 128 == 64:
            if str(ram) in task['startTime']:
                print(ram)
                del task['startTime'][str(ram)]
                del task['finishTime'][str(ram)]
            
with open('./generated-workflow.json', 'w+') as result_json:
    json.dump(data, result_json, indent=4)    

192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960


2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320


2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
288

1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
185

3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832

2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320
448
576
704
832
960
1088
1216
1344
1472
1600
1728
1856
1984
2112
2240
2368
2496
2624
2752
2880
3008
192
320


In [23]:
for ram in list(range(128, 3072 ,64)):
    if ram % 128 == 0:
        print('"' + str(ram) + '": ' + str('{0:.8f}'.format(0.00000021 * ram / 128)) + ',')

"128": 0.00000021,
"256": 0.00000042,
"384": 0.00000063,
"512": 0.00000084,
"640": 0.00000105,
"768": 0.00000126,
"896": 0.00000147,
"1024": 0.00000168,
"1152": 0.00000189,
"1280": 0.00000210,
"1408": 0.00000231,
"1536": 0.00000252,
"1664": 0.00000273,
"1792": 0.00000294,
"1920": 0.00000315,
"2048": 0.00000336,
"2176": 0.00000357,
"2304": 0.00000378,
"2432": 0.00000399,
"2560": 0.00000420,
"2688": 0.00000441,
"2816": 0.00000462,
"2944": 0.00000483,


In [24]:
for ram in list(range(128, 3072 ,64)):
    if ram % 128 == 0:
        print('"' + str(ram) + '",')

"128",
"256",
"384",
"512",
"640",
"768",
"896",
"1024",
"1152",
"1280",
"1408",
"1536",
"1664",
"1792",
"1920",
"2048",
"2176",
"2304",
"2432",
"2560",
"2688",
"2816",
"2944",


In [34]:
df_x = get_x_with_time()
df_x[(df_x['ram_mapped'] == 18.0) & (df_x['task_type'] == 7) & (df_x['workflow_size_mapped'] == 1.0)]

,ram_mapped,lambda_time_100ms_rounded,task_count_log,all_tasks_count_log,task_type,min_size_in_ln,max_size_in_ln,avg_size_in_ln,count_in_ln,sum_in_ln,min_size_out_ln,max_size_out_ln,avg_size_out_ln,count_out,sum_out_ln,exec_file_size_ln,workflow_size_mapped
7336,18.0,102.0,0.60206,2.970347,7,16.357402,16.357402,16.357402,1.098612,17.456015,15.063245,15.063245,15.063245,1,15.063245,15.433209,1.0
7574,18.0,50.0,0.60206,2.970347,7,16.357402,16.357402,16.357402,0.000000,16.357402,13.811694,13.811694,13.811694,1,13.811694,15.433209,1.0
7812,18.0,101.0,0.60206,2.970347,7,16.357402,16.357402,16.357402,1.098612,17.456015,15.063245,15.063245,15.063245,1,15.063245,15.433209,1.0
8050,18.0,49.0,0.60206,2.970347,7,16.357402,16.357402,16.357402,0.000000,16.357402,13.811694,13.811694,13.811694,1,13.811694,15.433209,1.0
2224,18.0,51.0,0.60206,2.970347,7,16.357402,16.357402,16.357402,0.000000,16.357402,13.826876,13.826876,13.826876,1,13.826876,15.433209,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2567,18.0,110.0,0.60206,2.970347,7,16.357402,16.357402,16.357402,1.098612,17.456015,15.063245,15.063245,15.063245,1,15.063245,15.433209,1.0
2576,18.0,53.0,0.60206,2.970347,7,16.357402,16.357402,16.357402,0.000000,16.357402,13.826876,13.826876,13.826876,1,13.826876,15.433209,1.0
2577,18.0,45.0,0.60206,2.970347,7,16.357402,16.357402,16.357402,0.000000,16.357402,13.654701,13.654701,13.654701,1,13.654701,15.433209,1.0
2579,18.0,54.0,0.60206,2.970347,7,16.357402,16.357402,16.357402,0.000000,16.357402,13.811694,13.811694,13.811694,1,13.811694,15.433209,1.0


In [28]:
def find_predecessor(workflow, the_task):
    predecessors = []
    for task in workflow['tasks']:
        if any(x in task['outs'] for x in the_task['ins']):
            predecessors.append(task)
    return predecessors

In [152]:
df_results = pd.read_csv('./results-montage.csv')


In [106]:
df_x = get_x()
model = tf.keras.models.load_model('./best-model-more-data')
getPredictedExecutionTime(1536, 'mBgModel', 1.0, df_x, model, 0)

in method
In else


256.5726

In [174]:
df_results[df_results['task_type'] == 8][['task_type', 'task_id']]

,task_type,task_id


In [9]:
df_results = df_results[df_results['successed'] == 0]
df_y = df_results['lambda_time_100ms_rounded']
del df_results['lambda_time_100ms_rounded']
del df_results['download_time_100ms_rounded_normalized']
del df_results['ram']
del df_results['execution_time_100ms_rounded']
del df_results['execution_time_100ms_rounded_normalized']
del df_results['upload_time_100ms_rounded']
del df_results['upload_time_100ms_rounded_normalized']
del df_results['download_time_100ms_rounded']
del df_results['task_id']
del df_results['successed']
del df_results['lambda_time_100ms_rounded_normalized']
del df_results['workflow']
del df_results['workflow_size']
df_x = df_results

In [11]:
df_x = df_results


In [90]:
df_x[(df_x['ram_mapped'] == 19.0) & (df_x['task_type'] == 3) & (df_x['workflow_size_mapped'] == 2.0)]

,ram_mapped,task_count_log,all_tasks_count_log,task_type,min_size_in_ln,max_size_in_ln,avg_size_in_ln,count_in_ln,sum_in_ln,min_size_out_ln,max_size_out_ln,avg_size_out_ln,count_out,sum_out_ln,exec_file_size_ln,workflow_size_mapped
81360,19.0,0.477121,2.791691,3,9.364605,10.293399,9.933168,0.693147,10.626315,7.441907,7.441907,7.441907,1,7.441907,13.733667,2.0
81443,19.0,0.477121,2.791691,3,9.364605,10.293399,9.933168,0.693147,10.626315,7.441907,7.441907,7.441907,1,7.441907,13.733667,2.0
81503,19.0,0.477121,2.791691,3,9.364605,10.293399,9.933168,0.693147,10.626315,7.441907,7.441907,7.441907,1,7.441907,13.733667,2.0
81979,19.0,0.477121,2.791691,3,9.364605,10.293399,9.933168,0.693147,10.626315,7.441907,7.441907,7.441907,1,7.441907,13.733667,2.0
82062,19.0,0.477121,2.791691,3,9.364605,10.293399,9.933168,0.693147,10.626315,7.441907,7.441907,7.441907,1,7.441907,13.733667,2.0


In [94]:
ids = list(df_x[(df_x['ram_mapped'] == 19.0) & (df_x['task_type'] == 0) & (df_x['workflow_size_mapped'] == 2.0)].index.values) 

In [95]:
ids

[81328,
 81333,
 81334,
 81342,
 81343,
 81351,
 81353,
 81357,
 81362,
 81364,
 81369,
 81371,
 81378,
 81396,
 81398,
 81402,
 81410,
 81416,
 81419,
 81420,
 81424,
 81437,
 81444,
 81448,
 81463,
 81465,
 81470,
 81474,
 81481,
 81482,
 81487,
 81488,
 81489,
 81492,
 81521,
 81524,
 81526,
 81531,
 81561,
 81568,
 81573,
 81576,
 81586,
 81590,
 81591,
 81595,
 81600,
 81602,
 81617,
 81631,
 81636,
 81652,
 81653,
 81657,
 81665,
 81670,
 81688,
 81708,
 81710,
 81717,
 81719,
 81749,
 81750,
 81761,
 81769,
 81776,
 81781,
 81783,
 81784,
 81808,
 81810,
 81815,
 81832,
 81837,
 81838,
 81842,
 81845,
 81847,
 81856,
 81868,
 81869,
 81878,
 81884,
 81893,
 81906,
 81917,
 81918,
 81935,
 81940,
 81941,
 81947,
 81952,
 81953,
 81961,
 81962,
 81970,
 81972,
 81976,
 81981,
 81983,
 81988,
 81990,
 81997,
 82015,
 82017,
 82021,
 82029,
 82035,
 82038,
 82039,
 82043,
 82056,
 82063,
 82067,
 82082,
 82084,
 82089,
 82093]

In [63]:
df_x.iloc[random.choice(ids)].values

array([17.        ,  2.29666519,  2.49136169,  1.        ,  5.62401751,
       15.24348415, 15.01932077,  1.60943791, 16.62875868,  5.56068163,
        5.56068163,  5.56068163,  1.        ,  5.56068163, 13.52333842,
        3.        ])

In [21]:
df_y[81443]

127.0

In [22]:
df_y[81503]

128.0

In [23]:
df_y[81979]

125.0

In [88]:
el = df_x.loc[[random.choice(ids)]].values

In [89]:
el

array([[19.        ,  0.47712125,  2.79169065,  3.        ,  9.36460533,
        10.29339899,  9.93316771,  0.69314718, 10.62631489,  7.44190673,
         7.44190673,  7.44190673,  1.        ,  7.44190673, 13.7336669 ,
         2.        ]])

In [92]:
model = tf.keras.models.load_model('./best-model-07.01')

In [144]:
el = random.choice(ids)

In [145]:
model.predict(df_x.loc[[random.choice(ids)]].values)

array([[150.99661]], dtype=float32)

In [146]:
df_y[el]

141.0

In [199]:
ram_list = list(range(128, 3072 ,64))
128	$0.0000000021
512	$0.0000000083

0.000000834
1024	$0.0000000167
1536	$0.0000000250
2048	$0.0000000333

In [208]:
0.0000166667 * 512/1024 /10

8.33335e-07

In [211]:
for ram in ram_list:      
    price_per_100ms = 0.0000166667 * ram/1024 /10
    print("\"" + str(ram) + "\": {:.8f},".format(price_per_100ms))

"128": 0.00000021,
"192": 0.00000031,
"256": 0.00000042,
"320": 0.00000052,
"384": 0.00000063,
"448": 0.00000073,
"512": 0.00000083,
"576": 0.00000094,
"640": 0.00000104,
"704": 0.00000115,
"768": 0.00000125,
"832": 0.00000135,
"896": 0.00000146,
"960": 0.00000156,
"1024": 0.00000167,
"1088": 0.00000177,
"1152": 0.00000188,
"1216": 0.00000198,
"1280": 0.00000208,
"1344": 0.00000219,
"1408": 0.00000229,
"1472": 0.00000240,
"1536": 0.00000250,
"1600": 0.00000260,
"1664": 0.00000271,
"1728": 0.00000281,
"1792": 0.00000292,
"1856": 0.00000302,
"1920": 0.00000313,
"1984": 0.00000323,
"2048": 0.00000333,
"2112": 0.00000344,
"2176": 0.00000354,
"2240": 0.00000365,
"2304": 0.00000375,
"2368": 0.00000385,
"2432": 0.00000396,
"2496": 0.00000406,
"2560": 0.00000417,
"2624": 0.00000427,
"2688": 0.00000438,
"2752": 0.00000448,
"2816": 0.00000458,
"2880": 0.00000469,
"2944": 0.00000479,
"3008": 0.00000490,


In [45]:
def custom_loss_function(y_actual, y_predicted):
    mask = tf.cast(tf.math.greater_equal(y_actual, y_predicted), tf.float32)
    mask = mask * 0.1 + 1
    y_actual = tf.math.multiply(y_actual, mask)
    return K.mean(K.square(K.abs(y_actual - y_predicted)))


In [46]:
actual = tf.constant([1.0, 3.0])
predicted = tf.constant([ 2.0, 1.0])
custom_loss_function(actual, predicted)

<tf.Tensor: shape=(), dtype=float32, numpy=3.1450005>